In [1]:
import pandas as pd
import spacy

In [2]:
import time

In [3]:
df_raw = pd.read_csv('../Data/review.csv')

In [4]:
df_raw.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,8aoJJdKEO3ypoZNszpPu7Q,bGgAL09pxLnV_FFgR4ZADg,ZBE-H_aUlicix_9vUGQPIQ,5.0,0,0,0,We had my Mother's Birthday Party here on 10/2...,2016-11-09 20:07:25
1,J5NOCLdhuhor7USRhtYZ8w,pFCb-1j6oI3TDjr26h2cJQ,e-YnECeZNt8ngm0tu4X9mQ,4.0,0,0,0,Good Korean grill near Eaton Centre. The marin...,2015-12-05 05:06:43
2,PXiLWAYRt3xnHaJ8MB4rzw,mEzc6LeTNiQgIVsq3poMbg,j7HO1YeMQGYo3KibMXZ5vg,5.0,2,1,3,Was recommended to try this place by few peopl...,2014-10-11 05:16:15
3,VrLarvxZYJm74yAqtpe9PQ,o-zUN2WEZgjQS7jnNsec0g,7e3PZzUpG5FYOTGt3O3ePA,3.0,0,0,0,Ambience: Would not expect something this nice...,2016-07-25 03:45:26
4,C1CUpidlVFprUCkApqzCmA,Wlx0iBXJvk4x0EeOt2Bz1Q,vuHzLZ7nAeT-EiecOkS5Og,1.0,11,0,3,Absolutely the WORST pool company that I have ...,2016-04-11 18:49:11


In [7]:
df = df_raw.copy()

# 3.3 Adjcective Phrases

In [8]:
import random
from collections import Counter
random.seed(448)

In [9]:
df

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,8aoJJdKEO3ypoZNszpPu7Q,bGgAL09pxLnV_FFgR4ZADg,ZBE-H_aUlicix_9vUGQPIQ,5.0,0,0,0,We had my Mother's Birthday Party here on 10/2...,2016-11-09 20:07:25
1,J5NOCLdhuhor7USRhtYZ8w,pFCb-1j6oI3TDjr26h2cJQ,e-YnECeZNt8ngm0tu4X9mQ,4.0,0,0,0,Good Korean grill near Eaton Centre. The marin...,2015-12-05 05:06:43
2,PXiLWAYRt3xnHaJ8MB4rzw,mEzc6LeTNiQgIVsq3poMbg,j7HO1YeMQGYo3KibMXZ5vg,5.0,2,1,3,Was recommended to try this place by few peopl...,2014-10-11 05:16:15
3,VrLarvxZYJm74yAqtpe9PQ,o-zUN2WEZgjQS7jnNsec0g,7e3PZzUpG5FYOTGt3O3ePA,3.0,0,0,0,Ambience: Would not expect something this nice...,2016-07-25 03:45:26
4,C1CUpidlVFprUCkApqzCmA,Wlx0iBXJvk4x0EeOt2Bz1Q,vuHzLZ7nAeT-EiecOkS5Og,1.0,11,0,3,Absolutely the WORST pool company that I have ...,2016-04-11 18:49:11
...,...,...,...,...,...,...,...,...,...
15295,qknwFVEh_0KSuexigYBI_A,YMGmyPOU65SMs4H60ltYiw,shIPnFoXrL3dFo5HLH1_HA,1.0,2,0,0,This was the worst experience ever. So much so...,2014-07-12 21:58:15
15296,Y-ZRoyAXCukBK1uK1ZcZCA,JLhOWQiWtGbr14K_KmoWxA,zPEYgVqJ2QNKi45FJi2jvg,5.0,0,0,0,We come here every time we hit Vegas! A giant ...,2018-11-10 21:38:49
15297,A8HdjBfhj3pgQuSbwNtDEw,6CoiKFDFXIACJZvv_I_8mQ,zPEYgVqJ2QNKi45FJi2jvg,1.0,0,1,0,As locals we used to the this place when it w...,2018-10-13 22:11:22
15298,2n1QdrYBRAAe6GKaxEV0jA,_fH4s3ls08eSl_PfX38KIA,etzDsNjkCyQBoJcU2a3U-g,5.0,0,0,0,The food was delicious. We were seated in 15 m...,2015-02-15 08:43:46


In [11]:
business_ids = df['business_id'].unique().tolist()
selected_ids = random.sample(business_ids, 20)

print(selected_ids)

['AktuBx1W7c3ZdzwuaOp8xg', 'dhBAQJLKPCI7nm3x0WjXTw', 'Mx8jCUdhaNbMgx_04DWHdA', 'vMpJzMFst_9GP4boeqWIRg', 'z8Em-bhZI3Mmspml7tj6tg', 'e-YnECeZNt8ngm0tu4X9mQ', 'AT5NZHqoLeP4rhWMM2OWBA', 'WO3L0pmtAO8ozspmaVdHIg', 'grvkjaJB9aorAMIjGD6XSg', 'QaFO4S6HFUu2NIbeu4OwCg', 'j7HO1YeMQGYo3KibMXZ5vg', 'NnxwkiBvSSyKeFiez0DDcQ', 'S6apFS5ghsQg69rcBvm2Qg', 'y3otpml08iLWdX5RCrWnbA', 'rv9T1qGHbh9eXqFpKEPy_A', 'Jol7cXrDmKKmu_V6qp4wMA', 'rN3gHTjqx5sOnoUpJ-6jyg', 'ukAUNyzUcBA4Su8UKjaJKg', 'z_RTK0MaxaRHRQ99hDL2QA', 'QxRKQ_Lzu4lMOejwVgcdAg']


In [12]:
reviews = df.loc[df['business_id'] == selected_ids[0]]

In [13]:
reviews

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
9318,o1I5vkgoK5JYqW0sbGJowA,rTIVsxvTEm5b94C2th5xQQ,AktuBx1W7c3ZdzwuaOp8xg,1.0,3,1,0,Gross!! I tried it cuz I woke across the stree...,2014-03-14 04:03:47
9359,x0m4DhqCX4Gqf-r7Am8VhQ,bC6kZANVn0-S19Y91drogA,AktuBx1W7c3ZdzwuaOp8xg,5.0,0,0,0,"fantastic, we tried 5 items and they were all ...",2016-06-12 04:35:12
9361,wnd6yVRVZEibaRJs4Jjgqg,HJj82f-csBI7jjgenwqhvw,AktuBx1W7c3ZdzwuaOp8xg,2.0,3,0,0,Not far from the house I really wanted to try ...,2011-12-01 07:09:23
9366,Jvm_diKSnZX2I8tYKbRN2Q,eytpwJXgg5nWU6jrlQ6qAA,AktuBx1W7c3ZdzwuaOp8xg,4.0,6,6,6,Dumpling is in their name and I didn't try one...,2014-01-18 14:58:34
9372,LPrp5XfAp86T33mHZ7Rkyw,7SijZ5lsoXWfCW3rKZ5BCg,AktuBx1W7c3ZdzwuaOp8xg,4.0,0,0,0,Came here for late lunch with my husband and s...,2015-12-21 22:31:27
...,...,...,...,...,...,...,...,...,...
11058,G0u7H4ZrRbkGrKc_rqT7Bw,H6XJFOG8IPH5UgAM-zS0YA,AktuBx1W7c3ZdzwuaOp8xg,1.0,2,1,0,Wow been going to this place for years and ton...,2016-03-06 03:39:39
11072,OXWdXZSW3-fDorhP3z91mg,cafRiSVWRXlxDEK2YNrJ6g,AktuBx1W7c3ZdzwuaOp8xg,4.0,1,0,0,We went here a few wks ago and decided to try ...,2011-02-01 23:36:36
11087,rOjidL0f6qOArHitejwT_w,2qFBGRvMfausQg5qqGlqeQ,AktuBx1W7c3ZdzwuaOp8xg,4.0,2,0,1,The food is really good here. It's faster to g...,2013-01-22 19:57:36
11130,ccHavGXMkhzjZ64VMpExmA,WP5CbpZweA-PyqW9W_YsyQ,AktuBx1W7c3ZdzwuaOp8xg,2.0,2,0,0,Great Chinese! One of the better options arou...,2015-02-01 03:32:10


In [14]:
nlp = spacy.load("en_core_web_sm")

In [15]:
adv_adj_pairs = []

for i in range(len(reviews)):
    doc = nlp(reviews.iloc[i]["text"])
    for i,token in enumerate(doc):
        if token.pos_ not in ('ADV'):
            continue
        for j in range(i+1,len(doc)):
            if doc[j].pos_  == 'ADJ':
                adv_adj_pairs.append((token,doc[j]))
                break
out = []

for pairs in adv_adj_pairs:
    item_1 = pairs[0]
    item_2 = pairs[1]
    out.append((str(item_1), str(item_2)))

c = Counter(out)
c.most_common(20)

[(('much', 'better'), 5),
 (('just', 'good'), 4),
 (('pretty', 'good'), 4),
 (('very', 'nice'), 3),
 (('again', 'little'), 3),
 (('really', 'close'), 2),
 (('so', 'great'), 2),
 (('very', 'good'), 2),
 (('Too', 'bad'), 2),
 (('again', 'Mongolian'), 2),
 (('very', 'affordable'), 2),
 (('again', 'new'), 2),
 (('so', 'new'), 2),
 (('very', 'tasty'), 2),
 (('when', 'good'), 2),
 (('here', 'single'), 2),
 (('so', 'happy'), 2),
 (('so', 'good'), 2),
 (('just', 'first'), 2),
 (('here', 'first'), 2)]